<a href="https://colab.research.google.com/github/Satyendersingh024/RAG-QA-Bot/blob/main/Copy_of_RAG_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Retrieval-Augmented Generation (RAG) Model for QA Bot**

In [ ]:
# Install the required packages (Pinecone, Cohere, and others)
!pip install pinecone-client cohere pandas scikit-learn

# Import necessary libraries
import pinecone
import cohere
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import zipfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Unzip the archive.zip
import zipfile
zip_path = '/content/drive/MyDrive/archive.zip'  # Replace with your actual file path
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/extracted')


In [ ]:
# Step 3: List the contents of the extracted folder
!ls /content/drive/MyDrive/extracted

credits.csv   links.csv        movies_metadata.csv  ratings_small.csv
keywords.csv  links_small.csv  ratings.csv


In [ ]:
# Load the CSV files into DataFrames
credits_df = pd.read_csv('/content/drive/MyDrive/extracted/credits.csv')
links_df = pd.read_csv('/content/drive/MyDrive/extracted/links.csv')
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
ratings_small_df = pd.read_csv('/content/drive/MyDrive/extracted/ratings_small.csv')
keywords_df = pd.read_csv('/content/drive/MyDrive/extracted/keywords.csv')
links_small_df = pd.read_csv('/content/drive/MyDrive/extracted/links_small.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/extracted/ratings.csv')


<ipython-input-18-f57cf595e65d>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


In [ ]:
# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')


In [ ]:
# Check if the index exists and create it if it doesn't
index_name = 'sample-movies'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Use the dimension specified for your embeddings
        metric='cosine',  # Use cosine similarity or another appropriate metric
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [ ]:
# Get the index
index = pc.Index(index_name)


In [ ]:
# Initialize Cohere
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

In [ ]:
# Prepare the data
# Fill missing values
movies_metadata_df['overview'] = movies_metadata_df['overview'].fillna('')

In [ ]:
# Example function to get text embeddings
def get_embeddings(texts):
    response = cohere_client.embed(texts=texts)
    return response.embeddings

In [ ]:
import cohere
from tqdm import tqdm
import pandas as pd
import pickle

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

def get_embeddings_batch(texts, batch_size=10):
    embeddings = []
    num_batches = (len(texts) + batch_size - 1) // batch_size  # Calculate the number of batches

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings", total=num_batches):
        batch = texts[i:i + batch_size]
        try:
            response = cohere_client.embed(texts=batch)
            embeddings.extend(response.embeddings)
        except Exception as e:
            print(f"Error generating embeddings for batch starting at index {i}: {e}")
    return embeddings

def save_embeddings(embeddings, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(embeddings, file)

# Load the CSV file
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
texts = movies_metadata_df['overview'].fillna('').tolist()

# Process embeddings in smaller batches
batch_size = 10  # Smaller batch size to reduce API load
embeddings = get_embeddings_batch(texts, batch_size)

# Save the embeddings to a file
save_embeddings(embeddings, '/content/drive/MyDrive/embeddings.pkl')

print("Embeddings generation and saving completed.")


<ipython-input-28-b4b0606a02d5>:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
Generating embeddings:   0%|          | 6/4547 [00:00<02:45, 27.49it/s]

Error generating embeddings for batch starting at index 0: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 10: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 20: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API cal

Generating embeddings:   0%|          | 14/4547 [00:00<02:10, 34.65it/s]

Error generating embeddings for batch starting at index 80: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 90: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 100: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API c

Generating embeddings:   0%|          | 22/4547 [00:00<02:03, 36.77it/s]

...

Error generating embeddings for batch starting at index 45300: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45310: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45320: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1

Generating embeddings: 100%|█████████▉| 4546/4547 [02:09<00:00, 37.67it/s]

Error generating embeddings for batch starting at index 45380: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45390: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error generating embeddings for batch starting at index 45400: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1

Generating embeddings: 100%|██████████| 4547/4547 [02:09<00:00, 35.11it/s]


Error generating embeddings for batch starting at index 45460: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Embeddings generation and saving completed.


In [ ]:
import pinecone
import pickle

# Correct API key and environment (verify these from Pinecone dashboard)
api_key = 'bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6'
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=api_key)

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to the index using the correct host
index = pinecone.Index(index_name, host=index_host)

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Prepare data for upsert (movie IDs and embeddings)
movie_ids = movies_metadata_df['id'].tolist()
vectors = [(str(movie_id), embedding) for movie_id, embedding in zip(movie_ids, embeddings)]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(vectors[i : i + batch_size])
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.core.client.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
        print(e.body)
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

Upserting to Pinecone completed.


In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd

# Initialize Pinecone and Cohere
pinecone.init(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1') # Use pinecone.init instead of Pinecone class
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Connect to Pinecone index
index = pinecone.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = cohere_client.generate(
        model='command-xlarge-20220901',  # Choose the appropriate model, update if needed
        prompt=prompt,
        max_tokens=100
    )
    return response.generations[0].text

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [(str(movie_id), embedding) for movie_id, embedding in zip(movie_ids, embeddings)]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(vectors[i : i + batch_size])
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]

# Query Pinecone
top_results = query_pinecone(query_embedding)
retrieved_ids = [match.id for match in top_results.matches]

# Retrieve the corresponding movie overviews
retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()

# Prepare prompt for generative model
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
response = generate_response(prompt)
print("Generated Response:", response)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = cohere_client.generate(
        model='command-xlarge-20220901',  # Choose the appropriate model, update if needed
        prompt=prompt,
        max_tokens=100
    )
    return response.generations[0].text

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]

# Query Pinecone
top_results = query_pinecone(query_embedding)
retrieved_ids = [match.id for match in top_results.matches]

# Retrieve the corresponding movie overviews
retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()

# Prepare prompt for generative model
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
response = generate_response(prompt)
print("Generated Response:", response)


<ipython-input-53-f52ea578214d>:58: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


Upserting to Pinecone completed.


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
# Check if your Cohere API key is a trial key and has exceeded its usage limit.
# If so, you might need to upgrade to a production key for higher rate limits.
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = cohere_client.generate(
        model='command-xlarge-20220901',  # Choose the appropriate model, update if needed
        prompt=prompt,
        max_tokens=100
    )
    return response.generations[0].text

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."

# Check the number of API calls remaining before making the call
# Consider adding a retry mechanism with exponential backoff in case of rate limiting
remaining_calls = cohere_client.get_available_usage().remaining
if remaining_calls > 0:
    query_embedding

Upserting to Pinecone completed.


<ipython-input-54-cd6a8cdb6c0b>:60: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


AttributeError: 'Client' object has no attribute 'get_available_usage'

In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
# Check if your Cohere API key is a trial key and has exceeded its usage limit.
# If so, you might need to upgrade to a production key for higher rate limits.
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = cohere_client.generate(
        model='command-xlarge-20220901',  # Choose the appropriate model, update if needed
        prompt=prompt,
        max_tokens=100
    )
    return response.generations[0].text

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."

# Removed call to get_available_usage as it does not exist.
query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]

# Query Pinecone
top_results = query_pinecone(query_embedding)
retrieved_ids = [match.id for match in top_results]

Upserting to Pinecone completed.


<ipython-input-56-554428847025>:60: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = cohere_client.generate(
        model='command-xlarge-20220901',  # Choose the appropriate model, update if needed
        prompt=prompt,
        max_tokens=100
    )
    return response.generations[0].text

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]

# Query Pinecone
top_results = query_pinecone(query_embedding)
retrieved_ids = [match.id for match in top_results.matches]

# Retrieve the corresponding movie overviews
retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()

# Prepare prompt for generative model
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
response = generate_response(prompt)
print("Generated Response:", response)


Upserting to Pinecone completed.


<ipython-input-57-f52ea578214d>:58: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv')


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
import pinecone
import cohere
import pickle
import pandas as pd
import time

# Initialize Pinecone and Cohere
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6', environment='us-east-1')
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Define Pinecone index parameters
index_name = 'sample-movies'

# Verify if index exists or create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match this to your embedding dimensions
        metric='cosine'
    )

# Connect to Pinecone index
index = pc.Index(index_name)

def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False  # Set to True if you want to include vector values
    )
    return results

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    max_retries = 5
    retry_delay = 10  # seconds
    for attempt in range(max_retries):
        try:
            response = cohere_client.generate(
                model='command-xlarge-20220901',  # Choose the appropriate model
                prompt=prompt,
                max_tokens=100
            )
            return response.generations[0].text
        except cohere.TooManyRequestsError as e:
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"An error occurred while generating a response: {e}")
            break

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata with dtype option to avoid warnings
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert (movie IDs and embeddings)
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(
            vectors=vectors[i:i + batch_size],
            namespace="default"  # Use default namespace, or specify if using multiple namespaces
        )
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")

# Example Query
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
try:
    query_embedding = cohere_client.embed(texts=[query_text]).embeddings[0]
except cohere.TooManyRequestsError as e:
    print(f"Rate limit exceeded while generating embedding: {e}")
    # Implement additional handling if necessary
    query_embedding = None

if query_embedding:
    # Query Pinecone
    top_results = query_pinecone(query_embedding)
    retrieved_ids = [match.id for match in top_results.matches]

    # Retrieve the corresponding movie overviews
    retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()

    # Prepare prompt for generative model
    prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

    # Generate response
    response = generate_response(prompt)
    print("Generated Response:", response)


Upserting to Pinecone completed.
Rate limit exceeded while generating embedding: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


In [ ]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pinecone
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer

# Initialize Pinecone
api_key = 'bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6'
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=api_key, environment='us-east-1') # Use Pinecone class to initialize client
index = pc.Index(index_name)

# Initialize SentenceTransformer for local embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)
movie_ids = movies_metadata_df['id'].tolist()

# Prepare data for upsert
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        index.upsert(vectors=vectors[i:i + batch_size], namespace="default")
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")


Upserting to Pinecone completed.


In [ ]:
import pickle
import pandas as pd

# Load saved embeddings and check content
try:
    with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
        embeddings = pickle.load(file)
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    embeddings = []  # Set embeddings to an empty list if file not found

# Verify the type and dimensions of embeddings
print(f"Type of embeddings: {type(embeddings)}")

# Check if embeddings are loaded properly
if len(embeddings) == 0:
    print("Embeddings list is empty.")
else:
    print(f"Sample embedding: {embeddings[0]}")
    print(f"Number of embeddings: {len(embeddings)}")

# Load movie metadata
try:
    movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)
    print("Movie metadata columns:", movies_metadata_df.columns)
    print("Sample data:", movies_metadata_df.head())
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    movies_metadata_df = pd.DataFrame()  # Set to empty DataFrame if file not found

# Ensure movie_ids and embeddings have the same length
if len(movies_metadata_df) == 0 or len(embeddings) == 0:
    print("Movie metadata or embeddings are not loaded properly.")
else:
    movie_ids = movies_metadata_df['id'].tolist()

    if len(movie_ids) != len(embeddings):
        raise ValueError("The length of movie IDs and embeddings do not match.")

    # Prepare data for upsert
    vectors = [
        {
            "id": str(movie_id),
            "values": list(map(float, embedding)),  # Ensure embeddings are floats
            "metadata": {"id": str(movie_id)}
        } for movie_id, embedding in zip(movie_ids, embeddings)
    ]

    # Upsert embeddings to Pinecone in smaller batches with error handling
    batch_size = 10
    for i in range(0, len(vectors), batch_size):
        try:
            batch_vectors = vectors[i:i + batch_size]
            index.upsert(vectors=batch_vectors, namespace="default")
            print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
        except pinecone.exceptions.ForbiddenException as e:
            print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
        except Exception as e:
            print(f"An error occurred for batch starting at index {i}: {e}")

    print("Upserting to Pinecone completed.")


Type of embeddings: <class 'list'>
Embeddings list is empty.
Movie metadata columns: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Sample data:    adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [

In [ ]:
import cohere
import pickle
import pandas as pd

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

# Prepare texts for embedding
texts = movies_metadata_df['overview'].tolist()  # Assuming you want to embed the 'overview' column

# Generate embeddings
try:
    response = cohere_client.embed(texts=texts)
    embeddings = response.embeddings
    print(f"Generated {len(embeddings)} embeddings.")
except Exception as e:
    print(f"Error generating embeddings: {e}")
    embeddings = []

# Save embeddings to a pickle file
with open('/content/drive/MyDrive/embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)


Error generating embeddings: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


In [ ]:
import pinecone

# Initialize Pinecone
pc = pinecone.Pinecone(api_key='bf3b5924-0a6d-49b2-bd75-fa0ccd2d75c6')

# Define Pinecone index parameters
index_name = 'sample-movies'
index_host = 'https://sample-movies-blulkao.svc.aped-4627-b74a.pinecone.io'

# Connect to Pinecone index
index = pinecone.Index(index_name, host=index_host)

# Load saved embeddings
with open('/content/drive/MyDrive/embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Prepare data for upsert (movie IDs and embeddings)
movie_ids = movies_metadata_df['id'].tolist()
vectors = [
    {
        "id": str(movie_id),
        "values": embedding,  # Ensure embeddings are in the correct format
        "metadata": {"id": str(movie_id)}
    } for movie_id, embedding in zip(movie_ids, embeddings)
]

# Upsert embeddings to Pinecone in smaller batches with error handling
batch_size = 10
for i in range(0, len(vectors), batch_size):
    try:
        batch_vectors = vectors[i:i + batch_size]
        index.upsert(vectors=batch_vectors, namespace="default")
        print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")
    except pinecone.exceptions.ForbiddenException as e:
        print(f"ForbiddenException occurred for batch starting at index {i}: {e}")
    except Exception as e:
        print(f"An error occurred for batch starting at index {i}: {e}")

print("Upserting to Pinecone completed.")


Upserting to Pinecone completed.


In [ ]:
import cohere
import pandas as pd

# Initialize Cohere client
cohere_client = cohere.Client('mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF')

# Load movie metadata
movies_metadata_df = pd.read_csv('/content/drive/MyDrive/extracted/movies_metadata.csv', dtype=str)

# Define query function for Pinecone
def query_pinecone(query_embedding, top_k=5):
    """
    Query Pinecone to retrieve the top_k most similar vectors.

    :param query_embedding: Embedding of the query text.
    :param top_k: Number of top similar vectors to retrieve.
    :return: List of IDs and scores of the most similar vectors.
    """
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=False,  # Set to True if you want to include vector values
        namespace="default"
    )
    return results

# Generate query embedding
query_text = "Tell me about movies with a strong female lead and action-packed scenes."
try:
    response = cohere_client.embed(texts=[query_text])
    query_embedding = response.embeddings[0]
    print("Generated query embedding.")
except Exception as e:
    print(f"Error generating query embedding: {e}")
    query_embedding = []

# Query Pinecone
try:
    top_results = query_pinecone(query_embedding)
    retrieved_ids = [match.id for match in top_results.matches]
    print(f"Retrieved IDs: {retrieved_ids}")
except Exception as e:
    print(f"Error querying Pinecone: {e}")
    retrieved_ids = []

# Retrieve the corresponding movie overviews
if retrieved_ids:
    retrieved_texts = movies_metadata_df[movies_metadata_df['id'].isin(retrieved_ids)]['overview'].tolist()
    print("Retrieved movie overviews.")
else:
    retrieved_texts = []

# Prepare prompt for generative model
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    try:
        response = cohere_client.generate(
            model='large',  # Choose the appropriate model
            prompt=prompt,
            max_tokens=100
        )
        return response.generations[0].text
    except Exception as e:
        print(f"Error generating response: {e}")
        return ""

response = generate_response(prompt)
print("Generated Response:", response)


Error generating query embedding: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error querying Pinecone: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 15 Sep 2024 16:10:06 GMT', 'Content-Type': 'text/plain', 'Content-Length': '9', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Wrong API key', 'www-authenticate': 'Wrong API key', 'server': 'envoy'})
HTTP response body: Forbidden

Error generating response: status_code: 400, body: {'message': "invalid request: model 'packed-embed-english-v2.0' is not supported by the generate API"}
Generated Response: 


In [ ]:
def generate_response(prompt):
    """
    Generate a response using a local model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text.
    """
    response = model.encode([prompt])
    return response

# Prepare prompt for generating response
prompt = f"Based on the following movie overviews, answer the question:\n\n{retrieved_texts}\n\nQuestion: What are the common themes and genres of these movies?"

# Generate response
response = generate_response(prompt)
print("Generated Response:", response)


Generated Response: [[ 2.52061095e-02 -2.16245782e-02 -6.67258799e-02 -2.67860107e-02
  -2.04798803e-02  1.28051564e-01  1.62249040e-02 -4.83349897e-02
  -1.13886287e-02 -5.46882376e-02 -1.65482480e-02  5.58066228e-03
  -3.96384075e-02  2.27480195e-02  2.47919913e-02  1.10172406e-02
   7.79798776e-02 -7.22616836e-02  4.46233600e-02  4.77510989e-02
   4.76576947e-02  1.18645523e-02  3.80065441e-02  4.26585488e-02
  -9.59874317e-02  8.25551746e-04  9.22793821e-02  9.81113613e-02
  -1.21091172e-01 -1.14398897e-02  4.11729999e-02  1.49853006e-01
  -3.01929284e-02 -2.44478248e-02 -3.40669155e-02  8.32059700e-03
  -1.61290057e-02  4.63753529e-02 -3.26164328e-02 -4.47497051e-03
  -2.39828620e-02 -1.02971867e-02  4.52702902e-02 -1.05538024e-02
   2.62364205e-02 -5.94246499e-02 -7.83585832e-02  6.64470624e-03
  -7.83923045e-02 -7.26152807e-02 -6.34288117e-02  2.12332103e-02
  -7.80854300e-02  7.13305250e-02  3.76378261e-02 -8.42391104e-02
  -3.91940586e-02  1.63959116e-02  9.02236402e-02 -4.832

In [ ]:
import cohere

# Replace with your new API key
API_KEY = 'mmql8ySYIdfSLrDtp1cE1SsMnvmApfD2Es2TGRLF'

def generate_response(prompt):
    """
    Generate a response using Cohere's generative model.

    :param prompt: The prompt for which the response is to be generated.
    :return: Generated response text or an error message.
    """
    try:
        cohere_client = cohere.Client(API_KEY)
        response = cohere_client.generate(
            model='command-xlarge-nightly',  # Changed to a generative model
            prompt=prompt,
            max_tokens=100,
            stop_sequences=["\n"]
        )
        if response and response.generations:
            return response.generations[0].text.strip()
        else:
            return "No response generated."
    except cohere.error.CohereAPIError as e: # Changed to the correct exception class
        return f"API Error: {e.message}"
    except Exception as e:
        return f"General Error: {e}"

# Sample prompt
prompt = "Explain the concept of Retrieval-Augmented Generation (RAG) in a simple manner."

# Generate response
response_text = generate_response(prompt)
print("Generated Response:", response_text)

Generated Response: Retrieval-Augmented Generation (RAG) is a fancy way of saying that a computer system can find information from a large collection of data and then use that information to generate a helpful response or answer. Imagine you have a super-smart robot friend who can quickly search through thousands of books in a library and then use what it finds to have a conversation with you.


finally we made it huhhh!😤